In [25]:
import pandas as pd
import json
from random import randint

In [3]:
def jsonl2df(path):
    with open(path) as fh:
        lines = fh.read().splitlines()

    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']


    df_inter['json_element'].apply(json.loads)

    return pd.json_normalize(df_inter['json_element'].apply(json.loads))

artists = jsonl2df("datav2/artists.jsonl")
tracks = jsonl2df("datav2/tracks.jsonl")

In [4]:
prepared_data = pd.merge(tracks,artists, left_on="id_artist", right_on="id")
prepared_data = prepared_data.dropna()

In [30]:
# zwraca dataframe(id, group)
def base_model(data: pd.DataFrame):
    track_times = data[['id_x', 'duration_ms']]

    # mieszanie wierszy
    track_times.sample(frac=1).reset_index(drop=True)
    current_group = 0
    group_sum = 0

    result = []

    for index, row in track_times.iterrows():
        result.append((row['id_x'], current_group))
        group_sum += row['duration_ms']
        if group_sum >= 3600000:        # godzina
            group_sum = 0
            current_group += 1

    # ostatnia grupa mniej niż godzina
    if group_sum < 3600000:
        for i in range(len(result)):
            if result[-1-i][1] == current_group:
                result[-1-i] = (result[-1-i][0], randint(0,current_group-1))
            else:
                break

    result_df = pd.DataFrame(result, columns=['id', 'group'])
    return result_df

In [33]:
base_model(prepared_data)